## Column Description
<span style="font-size: 14px;">

**track_id**: The Spotify ID for the track

**artists**: The artists' names who performed the track. If there is more than one artist, they are separated by a ;

**album_name**: The album name in which the track appears

**track_name**: Name of the track

**popularity**: The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.

**duration_ms**: The track length in milliseconds

**explicit**: Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)

**danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable

**energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

**key**: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1

**loudness**: The overall loudness of a track in decibels (dB)

**mode**: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0

**speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks

**acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic

**instrumentalness**: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content

**liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live

**valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)

**tempo**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration

**time_signature**: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.

**track_genre**: The genre in which the track belongs
</span>

In [1]:
import helper as h
dir(h)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'descriptive_stats_extended',
 'is_int_or_float',
 'mean',
 'ms_to_minutes_seconds',
 'np',
 'number_of_entries',
 'pd']

In [2]:
arr = [2 , 3 , 4]
df = h.pd.DataFrame(arr)
print("my mean function:" + repr(h.mean(arr)) + " pandas mean function: " + repr(df.mean().iloc[0]))

my mean function:3.0 pandas mean function: np.float64(3.0)


In [3]:
df = h.pd.read_csv('dataset.csv', encoding='utf-8')#load the dataset as a pandas dataframe

h.number_of_entries(df)

df = df.drop(columns=['Unnamed: 0']) #remove index column in dataset as the data frame will have its own index column by default

df.dropna(inplace=True)#remove null values

Number of files: 114000
Total number of data entries = 21 columns * 114000 files = 2394000
Total number of data entries (excluding index) = 20 * 114000 = 2280000


In [4]:
#this data frame is a summary of the number of values in each column and their data type
summarydf = h.pd.DataFrame(columns=['Column', 'Data type', 'Size', 'Number of unique Values'])

for loop_count, column in enumerate(df.columns):
    summarydf.loc[loop_count] = [column, df[column].dtype, df[column].size, df[column].nunique()]   #populate the dataframe with data from dataset 
    
summarydf.set_index('Column', inplace=True)

summarydf

,Data type,Size,Number of unique Values
Column,,,
track_id,object,113999,89740
artists,object,113999,31437
album_name,object,113999,46589
track_name,object,113999,73608
popularity,int64,113999,101
duration_ms,int64,113999,50696
explicit,bool,113999,2
danceability,float64,113999,1174
energy,float64,113999,2083


In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,113999.0,33.238827,22.304959,0.000,17.0000,35.000000,50.0000,100.000
duration_ms,113999.0,228031.153387,107296.057716,8586.000,174066.0000,212906.000000,261506.0000,5237295.000
danceability,113999.0,0.566801,0.173543,0.000,0.4560,0.580000,0.6950,0.985
energy,113999.0,0.641383,0.251530,0.000,0.4720,0.685000,0.8540,1.000
key,113999.0,5.309126,3.559999,0.000,2.0000,5.000000,8.0000,11.000
loudness,113999.0,-8.258950,5.029357,-49.531,-10.0130,-7.004000,-5.0030,4.532
mode,113999.0,0.637558,0.480708,0.000,0.0000,1.000000,1.0000,1.000
speechiness,113999.0,0.084652,0.105733,0.000,0.0359,0.048900,0.0845,0.965
acousticness,113999.0,0.314907,0.332522,0.000,0.0169,0.169000,0.5975,0.996
instrumentalness,113999.0,0.156051,0.309556,0.000,0.0000,0.000042,0.0490,1.000


In [6]:
statsdf = h.descriptive_stats_extended(df) #my own version of describe which includes variance and range. It also adds a row a duration in minutes and seconds after duration in milliseconds
statsdf

,Column,Mean,Standard deviation,Variance,Min,Max,Range,Lower Quartile,Median,Upper Quartile
4,popularity,33.238827,22.304959,497.511199,0,100,100,17.0,35.0,50.0
5,duration_ms,228031.153387,107296.057716,11512444001.445007,8586,5237295,5228709,174066.0,212906.0,261506.0
6,duration minutes and seconds,3m and 48s,1m and 47s,191874m and 4s,0m and 8s,87m and 17s,87m and 8s,2m and 54s,3m and 32s,4m and 21s
7,danceability,0.566801,0.173543,0.030117,0.0,0.985,0.985,0.456,0.58,0.695
8,energy,0.641383,0.25153,0.063267,0.0,1.0,1.0,0.472,0.685,0.854
9,key,5.309126,3.559999,12.673594,0,11,11,2.0,5.0,8.0
10,loudness,-8.25895,5.029357,25.294436,-49.531,4.532,54.063,-10.013,-7.004,-5.003
11,mode,0.637558,0.480708,0.23108,0,1,1,0.0,1.0,1.0
12,speechiness,0.084652,0.105733,0.011179,0.0,0.965,0.965,0.0359,0.0489,0.0845
13,acousticness,0.314907,0.332522,0.110571,0.0,0.996,0.996,0.0169,0.169,0.5975


In [7]:
#this code block can be used to edit a data frame by its numerical values
running = True
edited_df = df
edited_statdf = statsdf.copy() #make a copy so we still keep the original as it was
columns = df.select_dtypes(include=['number']).columns #this variable is to ensure only numerical columns can be selected for the editing the dataset
selected_column = h.pd.Series(dtype='object')
selected_max = 0
selected_min = 0
filter_value = 0
greater_or_less = ""
editing_notes = ""
while running:
    if selected_column.empty:
        column_name = input("enter column to filter by " + repr(columns.values))
        
        if " " in column_name:  #if the user added a space after the column they want to filter by we can remove it using split and access the first word in the list 
            column_name = column_name.split()[0]

        if column_name in columns:
            selected_column = edited_statdf[edited_statdf['Column'] == column_name]
            editing_notes += ("Selected data by " + column_name)
        else:
            print("Please enter a valid column name ")
            continue

    if not selected_column.empty:
        selected_max = selected_column['Max'].values[0]
        selected_min = selected_column['Min'].values[0]
        
    if filter_value == 0:
        filter_value = input("please enter the filter value between MIN: " + repr(selected_min) + " and MAX: " + repr(selected_max))
        
    
    if greater_or_less == "":
        greater_or_less = input("Would you like to select for values greater (g) or less than (l) " + repr(filter_value) + "?")
    
        if greater_or_less == "g" or greater_or_less == "G":
            edited_df = edited_df[edited_df[column_name] >= float(filter_value)]
            editing_notes += (" >= " + filter_value + "\n")
        elif greater_or_less == "l" or greater_or_less == "L":
            edited_df = edited_df[edited_df[selected_column] <= float(filter_value)]
            editing_notes += (" <= " + filter_value + "\n")
        else:
            print("please enter g or l")
            continue

    print(editing_notes)
    
    continue_editing = input("Would you like to continue editing the dataset? y/n")

    if continue_editing == "y":
        selected_column = h.pd.Series(dtype='object')
        filter_value = 0
        greater_or_less = ""
        edited_statdf = h.descriptive_stats_extended(edited_df)
        editing_notes = ""
        continue
    elif continue_editing == "n":
        running = False
    else:
        print("Please enter a valid character (y/n)")
        continue


Selected data by popularity >= 77

Selected data by popularity >= 77
Selected data by popularity <= 80



In [8]:
edited_df

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
h.descriptive_stats_extended(edited_df)

c:\Users\hamis\repos\MAT510\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\hamis\repos\MAT510\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


ValueError: cannot convert float NaN to integer

In [21]:
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.express as px
import random

In [22]:
#this function creates a dash app layout for comparing
def compare_axes(data : h.pd.DataFrame, app : Dash): #enforce type of arguments to avoid ambiguity
    numerical_columns = data.select_dtypes(include=['number']).columns #this variable is to ensure only numerical columns can be selected for the graph axes
    categorical_columns = data.select_dtypes(include=['object', 'category', 'string']).columns
    using_index = False
    
    if len(categorical_columns) == 0:
        using_index = True
        categorical_columns = data['index'] = data.index  # Create a new column with the DataFrame index
    
    # Ensure the dataset passed in contains at least one categorical and two numerical columns
    if len(categorical_columns) < 1 or len(numerical_columns) < 2:
        raise ValueError("Data must contain at least one categorical (object) column and two numerical columns.")
    
    if not using_index:
        initial_category = random.choice(categorical_columns) if len(categorical_columns) > 0 else None #randomise the category column
 
    app.layout = html.Div([
        html.Div([
            html.Label("Select Category:"),
            dcc.Dropdown(categorical_columns, initial_category, id='dropdown-category-selection')
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
        html.Div([
            html.Label("Select Values Within Category Column:"),
            dcc.Dropdown(id='dropdown-value-selection', multi=True, value=[])
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
         html.Div([
            html.Label("X-Axis"),
            dcc.Dropdown(numerical_columns, numerical_columns[0], id='dropdown-x-selection')
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),  
        html.Div([
            html.Label("Y-Axis:"),
            dcc.Dropdown(numerical_columns, numerical_columns[1], id='dropdown-y-selection')
        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'marginRight': '20px', 'background-color': 'coral'}),
        html.Div([
            dcc.Checklist(options=[{'label': 'Include Regression Line', 'value': 'show'}], id='regression-toggle',value=[])  # Empty list means it's unchecked by default
        ], style={'marginTop': '20px', 'background-color': 'coral'}),
        dcc.Graph(id='graph-content')
    ])
    # Update the values dropdown based on the selected categorical column
    @callback(
        Output('dropdown-value-selection', 'options'),
        Output('dropdown-value-selection', 'value'),  # Output the random value
        Input('dropdown-category-selection', 'value')
    )
    def update_value_options(selected_category):
        if(using_index):
            return 0
        else:
            # previous_category = selected_category
            unique_values = df[selected_category].unique()
            formatted_values = [{'label': str(i), 'value': str(i)} for i in unique_values]
            options = random.sample([i['value'] for i in formatted_values], 3)
            return formatted_values, options
    
    #update graph values
    @callback(
        Output('graph-content', 'figure'),
        Input('dropdown-category-selection', 'value'),
        Input('dropdown-value-selection', 'value'),
        Input('dropdown-x-selection', 'value'),
        Input('dropdown-y-selection', 'value'),
        Input('regression-toggle', 'value')
    )
    def update_graph(category_column, selected_values, valuex, valuey, regression_toggle):
        if selected_values:
            dff = data[data[category_column].isin(selected_values)]
            
        trendline = "ols" if "show" in regression_toggle else None  #sometimes it takes a little time for the regression line to appear on first loading up the notebook
        return px.scatter(dff, valuex, valuey, color=category_column, trendline=trendline, trendline_color_override='red')
    return app.layout

In [ ]:
#this code block uses the dash module to visualize some of the data
df = h.pd.read_csv('dataset.csv')
df = df.drop(columns=['Unnamed: 0', 'track_id']) #remove index column

app = Dash()

app.layout = compare_axes(df, app)

if __name__ == '__main__':
    app.run(debug=True)
